# Objectives 🔍
Our aim in this dataset is to characterize the relation between citric acid and three different variables.

We are not interested in building a model that will predict the amount of citric acid.

for simplicity we will use only 2 variables to predict the citric acid concentration

----------------------


# Loading Essential Libraries 📚

In [ ]:
# we start first by importing essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import statsmodels.api as sma
import statsmodels.stats.api as sms

from statsmodels.compat import lzip
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.gofplots import qqplot

import scipy.stats as stats

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Reading Data Set 👓

In [ ]:
url = 'https://raw.githubusercontent.com/linahourieh/Wine_Quality_Multilinear_Reg/main/winequality-red.csv'
df_wine = pd.read_csv(url)

In [ ]:
df_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
df_wine.shape

(1599, 12)

In [ ]:
df_wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [ ]:
df_wine.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [ ]:
# define some colors
colors = ['#FB938F', '#F2CAC8', '#C36B85', '#FDBB75']


def colorFader(c1,c2,mix=0):
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

c1=colors[2] #blue
c2=colors[1] #green
n=500

c=[]
for x in range(n+1):
    c.append(colorFader(c1,c2,x/n))


In [ ]:
y_data = [df_wine['fixed acidity']]
x_data = ['fixed acidity']

fig = go.Figure()

for xd, yd, cls in zip(x_data,y_data, c):
  fig.add_trace(
      go.Box(
              y=yd,
              name=xd,
              jitter=0.5,
              whiskerwidth=0.2,
              fillcolor=cls,
              marker_size=2,
              marker_color = 'black',
              line_width=0.7)
             )
      
fig.update_layout(template="plotly_white",
                  width=600,
                  height=700,
                  font=dict(size=18))
              
fig.show()

In [ ]:
y_data = [df_wine['free sulfur dioxide']]
x_data = ['free sulfur dioxide']

fig = go.Figure()

for xd, yd, cls in zip(x_data,y_data, c):
  fig.add_trace(
      go.Box(
              y=yd,
              name=xd,
              jitter=0.5,
              whiskerwidth=0.2,
              fillcolor=cls,
              marker_size=2,
              marker_color = 'black',
              line_width=0.7)
             )
      
fig.update_layout(template="plotly_white",
                  width=600,
                  height=700,
                  font=dict(size=18))
              
fig.show()

In [ ]:
df1 = df_wine[['fixed acidity', 'free sulfur dioxide', 'citric acid']]

In [ ]:
q_hi_1  = df_wine["free sulfur dioxide"].quantile(0.99)
q_hi_2  = df_wine["fixed acidity"].quantile(0.99)
x4 = df1[(df_wine["free sulfur dioxide"] < q_hi_1) & (df_wine["fixed acidity"] < q_hi_2)]
x4

,fixed acidity,free sulfur dioxide,citric acid
0,7.4,11.0,0.00
1,7.8,25.0,0.00
2,7.8,15.0,0.04
3,11.2,17.0,0.56
4,7.4,11.0,0.00
...,...,...,...
1594,6.2,32.0,0.08
1595,5.9,39.0,0.10
1596,6.3,29.0,0.13
1597,5.9,32.0,0.12


# Linear Regression Assumptions ☁️

## Assumption 1: Linear Relationship between y and x 📈

Linear regression needs the relationship between the independent and dependent variables to be linear. Let's use a scatter_matrix; equivalent to pairplot; to check the relation of independent variables with the citric acid

---------------------------

In [ ]:
fig = go.Figure(data=go.Splom(
                dimensions=[dict(label='fixed acidity',
                                 values=df_wine['fixed acidity']),
                            dict(label='free sulfur dioxide',
                                 values=df_wine['free sulfur dioxide']),
                            dict(label='citric acid',
                                 values=df_wine['citric acid'])],
                diagonal_visible=False, # remove plots on diagonal
                marker=dict(color='#C36B85',
                            showscale=False, 
                            size=4,# colors encode categorical variables
                            line_color='white', line_width=0)
                ))

fig.update_layout(
    template="plotly_white",
    title='Scatter Matrix',
    title_x=0.5,
    width=1000,
    height=900,
    font=dict(size=18)
)
fig.show()

By looking at the plots we can see that `fixed acidity` form an accurately linear shape with the `citric acid`, although some outliers exist. However, `free sulfur dioxide` shows not linearity with `citric acid`. A linear model might be able to efficiently explain the data in terms of variability and or prediction accuracy.

Now rest of the assumptions require us to perform the regression before we can even check for them. So let's perform regression on it.

# Model Development 🛠

In [ ]:
# prepare the X for the model
X = df_wine[['fixed acidity', 'free sulfur dioxide']]
X2  = sma.add_constant(X)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [ ]:
# write the model
model = sma.OLS(df_wine['citric acid'], X2)

In [ ]:
# now we fit our data to the model (let the model do the calculations based on our data)
res = model.fit()

## Assumption 2: Check for Homoscedasticity 📊
Homoscedasticity means that the residuals have equal or almost equal variance across the regression line. By plotting the residuals against the predicted terms we can check that there should not be any pattern.

------------



### **Graphical Method**


We plot the residuals against the predicted values or the X. If there is a definite pattern (like linear or quadratic or funnel shaped) obtained from the scatter plot then heteroscedasticity is present.

In [ ]:
fig = go.Figure(
    layout=go.Layout(
        title="Residuals vs fitted values plot for homoscedasticity check",
        title_x=0.5,
        width=1000,
        height=800,
        font=dict(size=18),
        template="plotly_white",
        autosize=True,
        yaxis_title="Residuals",
        xaxis_title="Predicted Values")
    )


fig.add_trace(go.Scatter(x=res.fittedvalues,
                         y=res.resid,
                         showlegend=False,
                         mode='markers',
                         name='lines',
                         marker=dict(color=colors[2],size = 5)))
fig.add_shape(type="line",
    x0=-0.25, y0=0, x1=0.9, y1=0,
    line=dict(
        color=colors[3],
        width=4
    ))


fig.show()

we can see from the graph that there is a diamond shape presenting the residuals. Also, there are some values that are very far from the zero line. So, the assumption here is not validated   ❌






### **Statistical Tests**


  




**Goldfeld Quandt Test:**

$$\mathcal{H}_{0}:  Residuals\ are\ homoscedastic\ $$ 

$$\mathcal{H}_{1}:  Residuals\ are\ not\ homoscedastic\ $$

In [ ]:
name = ['F statistic', 'p-value']
goldfeld = sms.het_goldfeldquandt(res.resid, X2)
lzip(name, goldfeld)

[('F statistic', 0.883001381178545), ('p-value', 0.9603427033226377)]



**Breusch Pagan Test for Heteroscedasticity**:

$$\mathcal{H}_{0}:  Residuals\ variances\ are\ equal\ (Homoscedasticity)$$ 

$$\mathcal{H}_{1}:  Residuals\ variances\ are\ not\ equal\ (Heteroscedasticity)\ $$ 


In [ ]:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
name = ['F statistic', 'p-value']
test = sms.het_breuschpagan(res.resid, X2)
lzip(name, test)

[('F statistic', 5.118287189723859), ('p-value', 0.07737097297710564)]

In both test the p-value is more than 0.05 in Goldfeld Quandt Test and Breush Pagan Test, we accept their null hypothesis that error terms are homoscedastic.  ✅

## Assumption 3: Check for Normality of residuals 🧮



----------------

In [ ]:
color = ['#C36B85']

hist_data = [res.resid]
group_labels = ['distribution of plot'] # name of the dataset

fig = ff.create_distplot(hist_data, 
                         group_labels,
                         bin_size=0.01,
                         colors= color)

fig.update_layout(
        title="Normality of Residuals",
        title_x=0.5,
        width=800,
        height=700,
        font=dict(size=18),
        template="plotly_white",
        yaxis_title="",
        xaxis_title="Possible residual Values")

fig.show()

In [ ]:
qqplot_data = qqplot(res.resid, line='s').gca().lines

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x= qqplot_data[0].get_xdata(),
                         y= qqplot_data[0].get_ydata(),
                         mode='markers',
                         marker=dict(color=colors[2],size =8 )))


fig.add_trace(go.Scatter(x= qqplot_data[1].get_xdata(),
                         y= qqplot_data[1].get_ydata(),
                         showlegend=False,
                         mode='lines',
                         marker=dict(color=colors[1],size = 5)))




fig.update_layout(
        title="Quantile-Quantile Plot",
        title_x=0.5,
        width=800,
        height=700,
        font=dict(size=18),
        showlegend =False,
        template="plotly_white",
        yaxis_title="Sample Quantities",
        xaxis_title="Theoritical Quantities")


fig.show()
#py.iplot(fig, filename='normality-QQ')


**Anderson Darling Test for checking Normality of Errors:**

$$\mathcal{H}_{0}:  The\ residuals\ follows\ a\ specified\ distribution $$ 

$$\mathcal{H}_{1}:  The\ residuals\ doesn't\ follows\ a\ specified\ distribution $$ 


In [ ]:
anderson_results = stats.anderson(res.resid, dist='norm')
name = ['Overall p-value', 'p-value']
lzip(name,anderson_results)

[('Overall p-value', 3.9312759942238245),
 ('p-value', array([0.575, 0.654, 0.785, 0.916, 1.089]))]

- The distribution plot shows somehow a bell shape, skewed to the right a little bit. But acceptable. ✅

- The Q-Q plot shows that most values are present on straight line. ✅

- In the test the p-value is more than 0.05 then, we accept the null hypothesis that residuals follows a normal distribution. ✅

## Assumption 4: Dropping Multicollinear Variables 🔻

In regression, multicollinearity refers to the extent to which independent variables are correlated. Multicollinearity affects the coefficients and p-values, but it does not influence the predictions, precision of the predictions, and the goodness-of-fit statistics. If your primary goal is to make predictions, and you don’t need to understand the role of each independent variable, you don’t need to reduce severe multicollinearity

-------------------------

In [ ]:
#Calculate the VIF and remove the features with VIF above 5 if you see fit to do so

s = df_wine[['fixed acidity', 'free sulfur dioxide']]
junk = []
x = add_constant(s)
vif0 = pd.Series([variance_inflation_factor(x.values, i)  for i in range(x.shape[1])], index=x.columns)
vif = pd.Series([variance_inflation_factor(x.values, i)  for i in range(x.shape[1])], index=x.columns)

#Eliminating Multicollinearity
while max(vif[1:])>5:
    junk=[]
    indice = [i for i, q in enumerate(vif[1:]) if q == max(vif[1:])][0]
    junk = np.append(junk,vif.index[indice+1])
    x = x.drop(columns = junk)
    vif =  pd.Series([variance_inflation_factor(x.values, i)  for i in range(x.shape[1])], index=x.columns)

print(vif0)
print(vif)

const                  29.047430
fixed acidity           1.024226
free sulfur dioxide     1.024226
dtype: float64
const                  29.047430
fixed acidity           1.024226
free sulfur dioxide     1.024226
dtype: float64


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [ ]:
def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

In [ ]:
z = df_wine[['fixed acidity', 'free sulfur dioxide', 'citric acid']]
z = z.corr()

In [ ]:
fig = go.Figure(data=go.Heatmap(df_to_plotly(z),
                                colorscale = c))

fig.update_layout(
        title="Heat Map",
        title_x=0.5,
        width=800,
        height=700,
        font=dict(size=18),
        showlegend =False,
        template="plotly_white")
fig.show()

- VIF showed no multicollinearity ✅
- Heatmap shared the same result. Note that the `fixed acidity` showed some correlation with the `citric acid` unlike the `free sulfur dioxide`. ✅


## Assumption 5: No Autocorrelation of Residuals 🔗

Linear regression model assumes that error terms are independent. This means that the error term of one observation is not influenced by the error term of another observation. In case it is not so, it is termed as autocorrelation.


**Durbin Watson test is used to check for autocorrelation:**

$$\mathcal{H}_{0}:  Autocorrelation\ is\ absent\  $$ 

$$\mathcal{H}_{1}:  Autocorrelation\ is\ present\ $$ 

In [ ]:
from statsmodels.stats.stattools import durbin_watson
durbin_watson(res.resid)

1.501533889724062

The value of the statistic will lie between 0 to 4. A value between 1.8 and 2.2 indicates no autocorrelation. A value less than 1.8 indicates positive autocorrelation and a value greater than 2.2 indicates negative autocorrelation

**Ljungbox test to ensure absence of autocorrelation:**

$$\mathcal{H}_{0}:  Autocorrelation\ is\ absent\  $$ 

$$\mathcal{H}_{1}:  Autocorrelation\ is\ present\ $$ 

In [ ]:
from statsmodels.stats import diagnostic as diag
min(diag.acorr_ljungbox(res.resid , lags = 40)[1])

8.591468574912609e-62

- Durbin test indicates positive autocorrelation. ❌
- Ljungbox showed no autocorrelation. ✅

# Evaluation 📍

In [ ]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            citric acid   R-squared:                       0.453
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     660.9
Date:                Thu, 10 Feb 2022   Prob (F-statistic):          7.96e-210
Time:                        10:30:17   Log-Likelihood:                 829.60
No. Observations:                1599   AIC:                            -1653.
Df Residuals:                    1596   BIC:                            -1637.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.3733    

## Overall Model Accuarcy:

This is evaluated by R-squared. R2 = 0.45 or 45%. THus, our model is **NOT** good enough to deploy on unseen data.

## Model Significance

Our Model:

$$\mathcal{Y}_{citric acid}= 0.075 {x}_{fixed acidity} +  0.0008 {x}_{free sulfur dioxide} - 0.33  $$

In order to prove that our linear model is statistically significant, we have to perform hypothesis testing for every β. Let us asume that:
$$\mathcal{H}_{0}: β_{1} = 0 $$
$$\mathcal{H}_{1}: β_{1} ≠ 0 $$
Simply, if β1 = 0 then the model shows no association between both variables 
$$\mathcal{Y}_{}= β_{0} + ε $$





To test the coefficient’s null hypothesis we will be using the t statistic. Look at the P>| t | column. These are the p-values for the t-test. In short, if they are less than the desired significance (commonly .05), you reject the null hypothesis. Otherwise, you fail to reject the null and therefore should toss out that independent variable.

Above, assuming a significance value of 0.05, our P-Value of 0.000 is much lower than a significance. Therefore, we reject the null hypothesis that the coefficient is equal to 0 and conclude that` fixed acidity` and `free sulfur dioxide` is an important independent variable to utilize.